## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']].copy()

In [4]:
# Create a list of at least 10 column names to use as X data
column_list = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[column_list].copy()

# Show the data types for X_df
X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

from sklearn.preprocessing import OneHotEncoder

overtime_encoder = OneHotEncoder(drop='first',  sparse_output=False)

# Fit the encoder to the training data
overtime_encoded = overtime_encoder.fit(X_train[['OverTime']])

In [7]:
def encode_categorical(X_data):
    # Separate numeric columns
    X_data_numeric = X_data.select_dtypes(include='number').reset_index(drop=True)

    overtime_feature_names = overtime_encoder.get_feature_names_out(['OverTime'])
    overtime_encoder_df = pd.DataFrame(overtime_encoder.transform(X_data[['OverTime']]), columns=overtime_feature_names)

    #Concat dfs tpgether
    X_data_encoded = pd.concat([X_data_numeric, overtime_encoder_df], axis=1)

    return X_data_encoded

In [8]:
X_train_encoded = encode_categorical(X_train)
X_test_encoded = encode_categorical(X_test)

In [9]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train_encoded)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train_encoded)
X_test_scaled = X_scaler.transform(X_test_encoded)

In [10]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)


# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = department_encoder.transform(y_train[['Department']])
y_test_department = department_encoder.transform(y_test[['Department']])

y_train_department

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [11]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(drop='first', sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition = attrition_encoder.transform(y_test[['Attrition']])

y_train_attrition

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [1.]])

## Part 2: Create, Compile, and Train the Model

In [12]:
# Find the number of columns in the X training data.
num_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,), name='input')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)


In [13]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)


In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attribution_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden)

In [15]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            704 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attribution_hidden        │ (None, 32)             │          4,128 │ shared2[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 1)              │             33 │ attribution_hidden[0]… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,412 (68.02 KB)

 Trainable params: 17,412 (68.02 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model
model.fit(X_train_scaled,
          {'department_output' : y_train_department, 'attrition_output': y_train_attrition},
          epochs=100,
          batch_size=32,
          validation_split=0.2
          )

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - attrition_output_accuracy: 0.6720 - attrition_output_loss: 0.6223 - department_output_accuracy: 0.5909 - department_output_loss: 0.9691 - loss: 1.5921 - val_attrition_output_accuracy: 0.8235 - val_attrition_output_loss: 0.4846 - val_department_output_accuracy: 0.6606 - val_department_output_loss: 0.7700 - val_loss: 1.2552
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8290 - attrition_output_loss: 0.4582 - department_output_accuracy: 0.6625 - department_output_loss: 0.7751 - loss: 1.2333 - val_attrition_output_accuracy: 0.8371 - val_attrition_output_loss: 0.4491 - val_department_output_accuracy: 0.6471 - val_department_output_loss: 0.7533 - val_loss: 1.2020
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8506 - attrition_output_loss: 0.4018 - department_output_accuracy: 0.6754 - department_output_loss: 0.7489 - loss: 1.1508 - val_attrition_output_accuracy: 0.8326 -

In [17]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': y_test_department, 'attrition_output': y_test_attrition})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8286 - attrition_output_loss: 0.9015 - department_output_accuracy: 0.5194 - department_output_loss: 3.8642 - loss: 4.7741 


[4.737591743469238,
 3.696730375289917,
 0.9867820143699646,
 0.8260869383811951,
 0.5135869383811951]

In [21]:
# Print the accuracy for both department and attrition

print(f"Attrition predictions accuracy: {test_results[3]}")
print(f"Department predictions accuracy: {test_results[4]}")

Attrition predictions accuracy: 0.8260869383811951
Department predictions accuracy: 0.5135869383811951


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Depending on whether you care more about False Negatives or False Positives, you may choose to focus on Recall or Precision. If you care more about False Negatives, you may want to focus on the Recall.  If you care more about False Positives, you may want to focus on the Precision. If you care about False Negatives and False Positive roughly equally, you may want to prioritize the F1-score, as it is a harmonic mean of precision and recall.  

   In the case of attrition, you may want to focus on recall as the False Negatives represent employees who the model predict will not leave but ultimately do. To minimize the number of employees who are incorrectly predicted to stay, you want the model to have a lower number of False Negatives and therefore a higher Recall. This would especially be the case if HR are interested in spotting employees who are likely to leave and take measures to discourage them from leaving.

2. For the attrition output layer, I chose the sigmoid activation function as it is a binary classsification problem. e.g attrition can be yes or no.  For the department output layer, I chose the softmax activation function as it is a multi-class classification e.g. there are three different departments.

3. To improve the model, you could look at ways to address the imbalanced dataset.  In the Attrition column, there are 1233 rows with No
and 237 with Yes.  In the Department column, there are 961 in Research & Development and 446 in Sales and 63 in Human Resource. As the dataset is imbalanced, you could look at resampling techniques such as Oversampling, Undersampling, SMOTE and SMOTEEN to see if that improves the results from the model.  

   Another way to improve the model would be to get more data to train with.  
   
   In addition, you could look at hyperparameters to see if they can be adjusted to improve the model's performance.


